# What do we want to incrementally evaluate ? 

1. Retrival 
2. Generation


# What are we going to modify ?

1. Embedding 
2. LLM models
3. Prompts and Chaining ( Planning ) 


In [2]:
!pip install cohere

   ---------------------------------------- 0.0/150.6 kB ? eta -:--:--
   ---------------------------------------- 150.6/150.6 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/497.2 kB ? eta -:--:--
   ----------------------- ---------------- 286.7/497.2 kB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 497.2/497.2 kB 6.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.2 MB 12.3 MB/s eta 0:00:01
   --- ------------------------------------ 0.2/2.2 MB 12.3 MB/s eta 0:00:01
   ----- ---------------------------------- 0.3/2.2 MB 2.4 MB/s eta 0:00:01
   ----- ---------------------------------- 0.3/2.2 MB 2.4 MB/s eta 0:00:01
   -------------------- ------------------- 1.1/2.2 MB 5.1 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.2 MB 5.2 MB/s eta 0:00:01
   ------------------------------ --------- 1.7/2.2 MB 5.6 MB/s eta 0:00:01
   ------------

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faster-whisper 0.9.0 requires tokenizers<0.15,>=0.13, but you have tokenizers 0.15.2 which is incompatible.
transformers 4.32.1 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.15.2 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from doc_loader import DocumentChunker
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import google.generativeai as genai 
import os
from langchain_experimental.agents import create_csv_agent
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from qdrant_client import QdrantClient
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.retrievers import MultiQueryRetriever
import cohere

# Initializing clients

In [2]:
from langchain_community.embeddings import CohereEmbeddings
load_dotenv()
co = cohere.Client(os.getenv("COHERE_API_KEY"))

In [12]:
#embed = co.embed(model="embed-english-v3.0")

In [3]:
response = co.embed(
  texts=['hello', 'goodbye'],
  model='embed-english-v3.0',
  input_type='classification'
)
print(response)

id='1d18a9ee-3c27-45ab-8743-82efea164538' embeddings=[[0.016036987, -0.008613586, -0.04925537, -0.070739746, 0.00039815903, -0.014801025, -0.019210815, 0.019805908, 0.009506226, 0.041748047, -0.036743164, 0.00014257431, 0.06347656, -0.03543091, 0.04458618, -0.030380249, 0.036621094, 0.048553467, 0.0054473877, 0.00737381, -0.020843506, 0.040802002, -0.04006958, 0.035003662, 0.05038452, 0.029266357, -0.004032135, -0.00097322464, -0.018692017, -0.039489746, 0.012863159, 0.0057296753, 0.018661499, 0.020599365, -0.0022678375, 0.089904785, -0.012641907, -0.026000977, 0.011993408, 0.004272461, 0.031051636, -0.030731201, -0.04058838, -0.023651123, -0.04321289, 0.00051116943, -0.017868042, -0.018478394, 0.07965088, -0.03125, 0.020248413, -0.014678955, 0.016921997, -0.0014009476, -0.03778076, 0.011566162, -0.07342529, -0.013412476, 0.010215759, 0.031433105, 0.045684814, 0.0042877197, 0.032226562, 0.04660034, -0.004322052, -0.019317627, 0.023162842, -0.013313293, 0.01940918, -0.015541077, 0.01641